In [1]:
from bs4 import BeautifulSoup
import urllib
import csv

In [18]:
def innerHTML(element):
    return element.decode_contents(formatter="html")

def get_name(body):
	return body.find('span', {'class':'jcn'}).a.string

def which_digit(html):
    mappingDict={'icon-ji':9,
                'icon-lk':8,
                'icon-nm':7,
                'icon-po':6,
                'icon-rq':5,
                'icon-ts':4,
                'icon-vu':3,
                'icon-wx':2,
                'icon-yz':1,
                'icon-acb':0,
                }
    return mappingDict.get(html,'')

def get_phone_number(body):
    i=0
    for item in body.find('p',{'class':'contact-info'}):
        i+=1
        if(i==2):
            phoneNo=''
            try:
                for element in item.find_all(class_=True):
                    classes = []
                    classes.extend(element["class"])
                    phoneNo+=str((which_digit(classes[1])))
            except:
                pass
            return phoneNo

def get_rating(body):
	rating = 0.0
	text = body.find('span', {'class':'star_m'})
	if text is not None:
		for item in text:
			rating += float(item['class'][0][1:])/10

	return rating

def get_rating_count(body):
	text = body.find('span', {'class':'rt_count'}).string

	# Get only digits
	rating_count =''.join(i for i in text if i.isdigit())
	return rating_count

def get_address(body):
	return body.find('span', {'class':'mrehover'}).text.strip()

def get_location(body):
	text = body.find('a', {'class':'rsmap'})
	if text == None:
		return
	text_list = text['onclick'].split(",")
	
	latitutde = text_list[3].strip().replace("'", "")
	longitude = text_list[4].strip().replace("'", "")
	
	return latitutde + ", " + longitude

In [19]:
page_number = 1
service_count = 1


In [20]:
fields = ['Name', 'Phone', 'Rating', 'Rating Count', 'Address', 'Location']
out_file = open('Logistics Baroda3.csv','w')
csvwriter = csv.DictWriter(out_file, delimiter=',', fieldnames=fields)

In [21]:
# Write fields first
csvwriter.writerow(dict((fn,fn) for fn in fields))

49

In [22]:
while True:

	# Check if reached end of result
	if page_number > 50:
		break

	url="https://www.justdial.com/Baroda/Logistic-Services/nct-10303689/page-%s" % (page_number)
	req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"}) 
	page = urllib.request.urlopen( req )
	# page=urllib2.urlopen(url)

	soup = BeautifulSoup(page.read(), "html.parser")
	services = soup.find_all('li', {'class': 'cntanr'})


	# Iterate through the 10 results in the page
	for service_html in services:

		# Parse HTML to fetch data
		dict_service = {}
		name = get_name(service_html)
		phone = get_phone_number(service_html)
		rating = get_rating(service_html)
		count = get_rating_count(service_html)
		address = get_address(service_html)
		location = get_location(service_html)
		if name != None:
			dict_service['Name'] = name
		if phone != None:
			print('getting phone number')
			dict_service['Phone'] = phone
		if rating != None:
			dict_service['Rating'] = rating
		if count != None:
			dict_service['Rating Count'] = count
		if address != None:
			dict_service['Address'] = address
		if location != None:
			dict_service['Address'] = location

		# Write row to CSV
		csvwriter.writerow(dict_service)

		print("#" + str(service_count) + " " , dict_service)
		service_count += 1

	page_number += 1

out_file.close()


#1  {'Name': 'Truck Guru', 'Rating': 4.6, 'Rating Count': '57', 'Address': 'SF- 12,  E-Tower Sun Meelan Complex,, Waghodia, Vadodara - 390019, Nr Ambe School, Waghodia Cross Road,'}
#2  {'Name': 'Anand Global', 'Rating': 4.0, 'Rating Count': '3', 'Address': 'B-4 ,Mangaldeep Complex, 0, Alkapuri, Vadodara - 390007, Masonic Hall'}
#3  {'Name': 'Organised Trans Solutions P..', 'Rating': 4.4, 'Rating Count': '13', 'Address': 'A 301 Saiganga Appartment, New VIP Road, Vip Road, Vadodara - 390022, Behind Earth Icon Mall'}
#4  {'Name': 'Narayan Powertech Pvt Ltd', 'Rating': 3.8, 'Rating Count': '12', 'Address': 'Padra Vadodara Road, Padra, Vadodara - 391440'}
#5  {'Name': 'Mahendra Logistic', 'Rating': 4.2, 'Rating Count': '13', 'Address': '20-B Ashtapad Society, Karelibaug, Vadodara - 390018, Near Karelibaug Water Tank'}
#6  {'Name': 'Nidhi Logistics', 'Rating': 4.9, 'Rating Count': '6', 'Address': '2nd Floor, 219 Sakar Complex, New VIP Road, Sardar Estate, Vadodara - 390019, Above super bekr

#51  {'Name': 'Delhi Rajasthan Transport C..', 'Rating': 3.5, 'Rating Count': '16', 'Address': 'B-10, Shreeji Market, Harni Road, Vadodara - 390006'}
#52  {'Name': 'Ecom Express Pvt Ltd', 'Rating': 3.7, 'Rating Count': '10', 'Address': '45, B/1, Sardar Estate, Road No 5, Ajwa Road, Vadodara - 390019'}
#53  {'Name': 'Surat Goods Transport Pvt L..', 'Rating': 3.7, 'Rating Count': '10', 'Address': '401/B, Alkapuri Arcade, R C Dutt Road, Alkapuri, Vadodara - 390007, Opposite Welcome Hotel'}
#54  {'Name': 'V Trans India Ltd', 'Rating': 3.3, 'Rating Count': '10', 'Address': 'Godwan No 5, J C Patel Estate, National Highway No 8, Dumad, Vadodara - 391740, Near Ankur Vidhyalay, Ranoli'}
#55  {'Name': 'Express Roadways Pvt Ltd', 'Rating': 4.4, 'Rating Count': '8', 'Address': '52, 3rd Floor, Dhanlaxmi Complex, Harni Road, Harni, Vadodara - 390022, Opposite Mangalya Hall, Near Airport Circle'}
#56  {'Name': 'Associated Road Carriers Li..', 'Rating': 3.6, 'Rating Count': '8', 'Address': 'R S Patel 

#101  {'Name': 'Perfect Mens Wear', 'Rating': 4.0, 'Rating Count': '2', 'Address': 'Bhayli, Shop 16 Vasana Bhayli Road, Alkapuri, Vadodara - 390007, Near Bright Day School'}
#102  {'Name': 'Blr Logistiks India Ltd', 'Rating': 4.0, 'Rating Count': '2', 'Address': '45 A, 2nd Floor, Dhanlaxmi Complex, Harni Road, Vadodara - 390006, Opposite Mangalayam Hall, Near Airport Circle'}
#103  {'Name': 'WW Shipping Agencies Pvt Lt..', 'Rating': 3.0, 'Rating Count': '2', 'Address': '107/108, Center Point, Alkapuri, Vadodara - 390007, Opposite Bank Of Baroda'}
#104  {'Name': 'Holisol Logistics Pvt Ltd', 'Rating': 3.0, 'Rating Count': '2', 'Address': 'Shop No7/8, Balaji Exotica, T.P13, Chhani, Vadodara - 390024, Near Saibaba Temple'}
#105  {'Name': 'Mks Global Logistics India ..', 'Rating': 5.0, 'Rating Count': '1', 'Address': 'T18b Indiabulls Mega Mall, Akota Road, Jetalpur Road, Vadodara - 390020, Near Dinesh Mill'}
#106  {'Name': 'Darcl Logistics Limited', 'Rating': 5.0, 'Rating Count': '1', 'Addr

#151  {'Name': 'Associated Road Carriers Lt..', 'Rating': 3.0, 'Rating Count': '1', 'Address': 'Plot No 41, Industrial Estate, Por, Vadodara - 391243, Raman Gamdi'}
#152  {'Name': 'Gati Ltd', 'Rating': 2.5, 'Rating Count': '1', 'Address': 'Shop No 244, Amitnagar Society, Vip Road, Vadodara - 390006, Near Reliance Fresh'}
#153  {'Name': 'Ashapura Trade & Transport ..', 'Rating': 2.0, 'Rating Count': '1', 'Address': 'Parcle Room, ST Depot Building, Sayaji Ganj, Vadodara - 390020, Opposite Railway Station'}
#154  {'Name': 'Allcargo Logistics Ltd', 'Rating': 2.0, 'Rating Count': '1', 'Address': '516, Siddharth Complex, R C Dutt Road, Alkapuri, Vadodara - 390007'}
#155  {'Name': 'Sepl Logistics', 'Rating': 2.0, 'Rating Count': '1', 'Address': '226, VIP View, VIP Road, Karelibaug, Vadodara - 390018'}
#156  {'Name': 'Shree Bhawani Logistics Pac..', 'Rating': 1.0, 'Rating Count': '1', 'Address': '318, 3rd Floor, Sakar Complex, New Vip Road, Ajwa Road, Vadodara - 390019, Near Baroda Super Baker

#201  {'Name': 'Anubhav Kargo Pvt. Ltd.', 'Rating': 0.0, 'Rating Count': '0', 'Address': 'Office: 18 B, Ground Floor, Nutan Bharat Society,, Opp.reliance Fresh,, Alkapuri, Vadodara - 390007, Alkapuri'}
#202  {'Name': 'Transit Point Logistics', 'Rating': 0.0, 'Rating Count': '0', 'Address': 'Main road, Jetalpur Road, Vadodara - 390020'}
#203  {'Name': 'Flomic Freight Services Pvt..', 'Rating': 0.0, 'Rating Count': '0', 'Address': 'Shop No 428, Atlantis K10/b, Sarabhai Road, Alkapuri, Vadodara - 390007, Near Genda Circle'}
#204  {'Name': 'Agility Logistics Pvt Ltd', 'Rating': 0.0, 'Rating Count': '0', 'Address': '507,Ocean Business Complex, Sarabhai Road, Alkapuri, Vadodara - 390007, Opposite Vadodara Central Mall,Near Genda Circle'}
#205  {'Name': 'Exim Transtrade (India) Pvt..', 'Rating': 0.0, 'Rating Count': '0', 'Address': '210,Second Floor,Atlantis K10b, Sarabhai Road, Alkapuri, Vadodara - 390023, Opposite Honest Restaurant'}
#206  {'Name': 'Shreeji Shipping', 'Rating': 0.0, 'Rating

#251  {'Name': 'Shree Sharp Logistics', 'Rating': 0.0, 'Rating Count': '0', 'Address': '13/ A, Maruti Estate, Golden Chowkdi, Harni Road, Vadodara - 390006, Near Halol Toll Tax'}
#252  {'Name': 'Umi Logistics', 'Rating': 0.0, 'Rating Count': '0', 'Address': 'B-185, Pramukh Swami Nagar, Harni Varasia Ring Road, Harni Road, Vadodara - 390006, Near  Nita Prasuti Gruh'}
#253  {'Name': 'Anita Logistic', 'Rating': 0.0, 'Rating Count': '0', 'Address': 'Janki Kutir, Near Baroda Club, Harni Road, Vadodara - 390006, Harni Kotli Road'}
#254  {'Name': 'All India Logistics', 'Rating': 0.0, 'Rating Count': '0', 'Address': 'Aashiyana Building, Khatriwad, Fatehpura, Vadodara - 390006'}
#255  {'Name': 'Bipin Transport Pvt Ltd', 'Rating': 0.0, 'Rating Count': '0', 'Address': '107, Vraj Complex, Airport Road, Harni, Vadodara - 390022, Old Harni Jakat Naka'}
#256  {'Name': 'Gobox Logistics', 'Rating': 0.0, 'Rating Count': '0', 'Address': '14, Poonam Complex, Waghodia Road, Vadodara - 390025, Near Kaladars